# Webpage topics extraction utility
### Created by: <br> <br> Varad S. Tupe <br> [Email](mailto:varad.s.tupe@gmail.com)<br>[LinkedIn](http://www.linkedin.com/in/varadtupe) <br> [GitHub](http://github.com/varadtupe) <br> [About Me](http://about.me/varadtupe)

This utility is designed to extract textual data from webpages and perform topic modelling on web page.

### Requirements:
    1. Python 3.6
    2. nltk
    3. genism
    4. pyLDAvis
    5. matplotlib
    6. urlib
    7. bs4 (beautiful soup)

### Utility Architecture
<img src="./flow.jpg">

### Classes:
#### Page_Loader
- This is class handles all the data extraction from given URL.
- text_from_html method returns the text data for given URL.

#### Text_Analzyer
- This class handles the data pre-processing and create the LDA model for topic extraction.

#### Topic_Modeler
- This class is more of an executor.
- Has 2 methods
- `process_lda` method handles one URL at time.
- `process_bulk_lda` method handles multiple URLs at once and provide topic modelling for all URLs all together.
- Both the methods would have 3 tries to fetch data from URL.
- The `visualize_topic` method uses `pyLDAvis` library in order to provide interactive visualitization of topics extracted in the process.

#### Note: Data extraction from some URLs might fail since `urlib` is not capable handling data form webpages which uses dynamic content loading using ajax call
Hence this utility can't handle data extraction from site like `amazon.com`. However it can handle data extraction from `ebay.com`.
In order to over come this we need to use more simulation utilites like `selenium` or write `spyders` with custom configuration.


In [1]:
import re
import gensim
from gensim import corpora
from nltk.stem.porter import PorterStemmer
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from gensim.models.ldamodel import LdaModel
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt


class Text_Analyzer():
    def __init__(self):
        self.reg_tokenizer = RegexpTokenizer(r'\w+')
        # use English stop words list
        self.en_stop = stopwords.words('english')
        # use self.p_stemmer of class PorterStemmer
        self.p_stemmer = PorterStemmer()
        self.lemmetizer = nltk.WordNetLemmatizer()
        self.corpus = ""
        self.dictionary = ""

    def remove_spl_char(self, inp_text):
        return re.sub("[^a-zA-Z ]","", inp_text)
    
    def reg_tokenize(self, raw_input):
        return self.reg_tokenizer.tokenize(raw_input)

    def remove_stop_word(self, tokens):
        return [i.lower() for i in tokens if not i.lower() in self.en_stop and len(i) > 2 ]
    
    def stemm_tokens(self, tokens):
        return [self.p_stemmer.stem(i) for i in tokens]
    
    def lemmetize_token(self, tokens):
        return [self.lemmetizer.lemmatize(token) for token in tokens]
    
    # turn our tokenized documents into a id <-> term dictionary
    def create_dictionary(self, tokens):
        self.dictionary = corpora.Dictionary(tokens)

    def create_corpus(self, tokens, dictionary):
        self.corpus = [dictionary.doc2bow(text) for text in tokens]
    
    def create_LDA_model(self):
        self.ldamodel = LdaModel(
            self.corpus,
            num_topics=10,
            id2word = self.dictionary,
            passes=30
            )

    def print_topics(self):
        print('Top 10 key words from the webpage')
        for i in self.ldamodel.show_topics(num_words=10, formatted=False)[0][1]:
            print(*i)

    def visualize_topic(self):
        pyLDAvis.enable_notebook()
        return pyLDAvis.gensim.prepare(
            self.ldamodel,
            self.corpus,
            self.dictionary
        )
    
    def process_lda(self, inp_text):
        text = self.remove_spl_char(inp_text)
        tokens = self.reg_tokenize(text)
        tokens = self.remove_stop_word(tokens)
        tokens = self.lemmetize_token(tokens)
        self.create_dictionary([tokens])
        self.create_corpus([tokens], self.dictionary)
        self.create_LDA_model()
    
    def process_bulk_lda(self, text_list):
        token_list = []
        for page in text_list:
            text = self.remove_spl_char(page)
            tokens = self.reg_tokenize(text)
            tokens = self.remove_stop_word(tokens)
            tokens = self.lemmetize_token(tokens)
            token_list.append(tokens)
        
        self.create_dictionary(token_list)
        self.create_corpus(token_list, self.dictionary)
        self.create_LDA_model()


In [2]:
import urllib.request

from bs4 import BeautifulSoup
from bs4.element import Comment


class Page_Loader():
    '''
    This class extract the text from any webpage
    '''
    def __init__(self):
        pass
    
    def tag_visible(self, element):
        if element.parent.name in ['style', 'script', 'head', 'title', 'meta', '[document]']:
            return False
        if isinstance(element, Comment):
            return False
        return True


    def text_from_html(self, url):
        body = urllib.request.urlopen(url).read()
        soup = BeautifulSoup(body, 'html.parser')
        texts = soup.findAll(text=True)
        visible_texts = filter(self.tag_visible, texts)  
        return u" ".join(t.strip() for t in visible_texts)


In [3]:
class Topic_Modeller():

    def __init__(self):
        self.pg_loader = Page_Loader()
        self.txt_analyzer = Text_Analyzer()
    def process_url(self, inp_url):
        num_retries = 3
        success_flag = False
        for try_num in range(num_retries):
            if not success_flag:
                try:    
                    text_data = self.pg_loader.text_from_html(inp_url)
                    success_flag = True
                    print('Success')
                    break
                except:
                    print('Failed to fetch data on try num #', try_num)
        if not success_flag:
            raise Exception('Error while fetching data for url: ' + inp_url)
        self.txt_analyzer.process_lda(text_data)
        self.txt_analyzer.print_topics()
        return self.txt_analyzer

    def process_bulk_url(self, inp_list):
        num_retries = 3
        text_data = []
        url_queue = inp_list.copy()
        
        for _ in range(num_retries):
            url_list = url_queue.copy()
            url_queue = []
            for url in url_list:
                try:
                    page_text = self.pg_loader.text_from_html(url)
                    text_data.append(page_text)
                except:
                    url_queue.append(url)
            if len(url_queue) == 0:
                break


        self.txt_analyzer.process_bulk_lda(text_data)
        
        if len(url_queue) > 0:
            print('Unable to fetch data from following URLS:')
            for url in url_queue:
                print(url)
        return self.txt_analyzer


In [4]:
tm = Topic_Modeller()

#### Topic modelling on DSLR camera listed on Ebay.com

In [5]:
result = tm.process_url('https://www.ebay.com/itm/Canon-EOS-5D-Mark-IV-MK-4-DSLR-Camera-Body-Only/162197189088')
result.visualize_topic()

Success
Top 10 key words from the webpage
shipping 0.00161368
new 0.00161359
item 0.00161271
eos 0.00161267
camera 0.00161243
window 0.00161225
canon 0.00161222
open 0.00161216
seller 0.00161198
digital 0.00161177


PreparedData(topic_coordinates=            Freq  cluster  topics         x    y
topic                                           
7      99.949008        1       1  0.097449 -0.0
9       0.005666        1       2 -0.010841  0.0
8       0.005666        1       3 -0.010855 -0.0
6       0.005666        1       4 -0.010852 -0.0
5       0.005666        1       5 -0.010837  0.0
4       0.005666        1       6 -0.010833  0.0
3       0.005666        1       7 -0.010786 -0.0
2       0.005666        1       8 -0.010816 -0.0
1       0.005666        1       9 -0.010808  0.0
0       0.005666        1      10 -0.010820  0.0, topic_info=     Category       Freq       Term      Total  loglift  logprob
term                                                            
349   Default  35.000000        new  35.000000  30.0000  30.0000
73    Default  34.000000     camera  34.000000  29.0000  29.0000
498   Default  33.000000   shipping  33.000000  28.0000  28.0000
77    Default  30.000000      canon  30.000000  27.0000  27.0000
485   Default  26.000000     seller  26.000000  26.0000  26.0000
363   Default  24.000000       open  24.000000  25.0000  25.0000
271   Default  22.000000       item  22.000000  24.0000  24.0000
168   Default  21.000000        eos  21.000000  23.0000  23.0000
538   Default  20.000000        tab  20.000000  22.0000  22.0000
607   Default  20.000000     window  20.000000  21.0000  21.0000
316   Default  18.000000       mark  18.000000  20.0000  20.0000
481   Default  17.000000        see  17.000000  19.0000  19.0000
144   Default  15.000000    digital  15.000000  18.0000  18.0000
158   Default  15.000000       ebay  15.000000  17.0000  17.0000
432   Default  14.000000   purchase  14.000000  16.0000  16.0000
250   Default  13.000000      image  13.000000  15.0000  15.0000
337   Default  13.000000      month  13.000000  14.0000  14.0000
216   Default  13.000000       free  13.000000  13.0000  13.0000
107   Default  12.000000  condition  12.000000  12.0000  12.0000
527   Default  11.000000      state  11.000000  11.0000  11.0000
383   Default  11.000000    payment  11.000000  10.0000  10.0000
381   Default  11.000000       past  11.000000   9.0000   9.0000
136   Default  11.000000   delivery  11.000000   8.0000   8.0000
252   Default  10.000000     import  10.000000   7.0000   7.0000
225   Default  10.000000       good  10.000000   6.0000   6.0000
228   Default   9.000000      great   9.000000   5.0000   5.0000
141   Default   9.000000     detail   9.000000   4.0000   4.0000
22    Default   9.000000     amount   9.000000   3.0000   3.0000
570   Default   9.000000     united   9.000000   2.0000   2.0000
510   Default   9.000000        slr   9.000000   1.0000   1.0000
...       ...        ...        ...        ...      ...      ...
498   Topic10   0.000161   shipping  33.890409  -2.4770  -6.4292
349   Topic10   0.000161        new  35.821596  -2.5325  -6.4293
271   Topic10   0.000161       item  22.304521  -2.0593  -6.4298
168   Topic10   0.000161        eos  21.338954  -2.0151  -6.4299
73    Topic10   0.000161     camera  34.856091  -2.5059  -6.4300
607   Topic10   0.000161     window  20.373475  -1.9690  -6.4301
77    Topic10   0.000161      canon  30.028572  -2.3570  -6.4301
363   Topic10   0.000161       open  24.235509  -2.1427  -6.4302
485   Topic10   0.000161     seller  26.166434  -2.2194  -6.4303
144   Topic10   0.000161    digital  15.545946  -1.6989  -6.4304
216   Topic10   0.000161       free  13.615010  -1.5663  -6.4305
538   Topic10   0.000161        tab  20.373472  -1.9695  -6.4306
316   Topic10   0.000161       mark  18.442566  -1.8699  -6.4306
481   Topic10   0.000161        see  17.477017  -1.8162  -6.4306
432   Topic10   0.000161   purchase  14.580490  -1.6350  -6.4307
527   Topic10   0.000161      state  11.683981  -1.4136  -6.4308
250   Topic10   0.000161      image  13.614993  -1.5666  -6.4308
158   Topic10   0.000161       ebay  15.545995  -1.6992  -6.4308
252   Topic10   0.000161     import  10.718467  -1.327

#### Topic modelling from REI blog regarding how to introduce your indoorsy friend to the outdoors

In [6]:
result = tm.process_url('http://blog.rei.com/camp/how-to-introduce-your-indoorsy-friend-to-the-outdoors/')
result.visualize_topic()

Success
Top 10 key words from the webpage
friend 0.00306862
rei 0.00306852
camping 0.00306807
coop 0.00306804
like 0.00306803
hike 0.003068
take 0.00306797
keep 0.0030679
time 0.0030679
flat 0.00306787


PreparedData(topic_coordinates=            Freq  cluster  topics         x    y
topic                                           
5      99.806452        1       1  0.031964 -0.0
9       0.021505        1       2 -0.003545 -0.0
8       0.021505        1       3 -0.003555 -0.0
7       0.021505        1       4 -0.003551 -0.0
6       0.021505        1       5 -0.003555 -0.0
4       0.021505        1       6 -0.003553 -0.0
3       0.021505        1       7 -0.003551  0.0
2       0.021505        1       8 -0.003549  0.0
1       0.021505        1       9 -0.003552 -0.0
0       0.021505        1      10 -0.003553  0.0, topic_info=     Category       Freq          Term      Total  loglift  logprob
term                                                               
105   Default  12.000000        friend  12.000000  30.0000  30.0000
228   Default   7.000000           rei   7.000000  29.0000  29.0000
158   Default   6.000000          like   6.000000  28.0000  28.0000
37    Default   5.000000       camping   5.000000  27.0000  27.0000
287   Default   5.000000          take   5.000000  26.0000  26.0000
147   Default   4.000000          keep   4.000000  25.0000  25.0000
120   Default   4.000000          hike   4.000000  24.0000  24.0000
294   Default   4.000000          time   4.000000  23.0000  23.0000
109   Default   4.000000          gear   4.000000  22.0000  22.0000
60    Default   4.000000          coop   4.000000  21.0000  21.0000
139   Default   3.000000      jennifer   3.000000  20.0000  20.0000
114   Default   3.000000         great   3.000000  19.0000  19.0000
65    Default   3.000000    davisflynn   3.000000  18.0000  18.0000
99    Default   3.000000          flat   3.000000  17.0000  17.0000
233   Default   3.000000         reply   3.000000  16.0000  16.0000
2     Default   3.000000      activity   3.000000  15.0000  15.0000
171   Default   3.000000         might   3.000000  14.0000  14.0000
142   Default   2.000000       journal   2.000000  13.0000  13.0000
168   Default   2.000000           may   2.000000  12.0000  12.0000
165   Default   2.000000         login   2.000000  11.0000  11.0000
254   Default   2.000000         short   2.000000  10.0000  10.0000
21    Default   2.000000          best   2.000000   9.0000   9.0000
66    Default   2.000000           day   2.000000   8.0000   8.0000
88    Default   2.000000        expert   2.000000   7.0000   7.0000
312   Default   2.000000         water   2.000000   6.0000   6.0000
155   Default   2.000000           let   2.000000   5.0000   5.0000
295   Default   1.000000           tip   1.000000   4.0000   4.0000
132   Default   1.000000      indoorsy   1.000000   3.0000   3.0000
273   Default   1.000000          star   1.000000   2.0000   2.0000
58    Default   1.000000  conversation   1.000000   1.0000   1.0000
...       ...        ...           ...        ...      ...      ...
105   Topic10   0.000306        friend  12.219093  -2.1496  -5.7865
228   Topic10   0.000306           rei   7.556348  -1.6691  -5.7866
37    Topic10   0.000306       camping   5.691259  -1.3858  -5.7867
60    Topic10   0.000306          coop   4.758740  -1.2068  -5.7867
158   Topic10   0.000306          like   6.623804  -1.5375  -5.7867
120   Topic10   0.000306          hike   4.758726  -1.2068  -5.7867
287   Topic10   0.000306          take   5.691281  -1.3858  -5.7867
147   Topic10   0.000306          keep   4.758722  -1.2069  -5.7868
294   Topic10   0.000306          time   4.758728  -1.2069  -5.7868
99    Topic10   0.000306          flat   3.826184  -0.9888  -5.7868
65    Topic10   0.000306    davisflynn   3.826183  -0.9888  -5.7868
2     Topic10   0.000306      activity   3.826186  -0.9888  -5.7868
139   Topic10   0.000306      jennifer   3.826178  -0.9888  -5.7868
109   Topic10   0.000306          gear   4.758737  -1.2069  -5.7868
114   Topic10   0.000306         great   3.826182  -0.9888  -5.7868
171   Topic10   0.000306         might   3.826189  -0.9888  -5.7868
142   Topic10   0.000306       journa

#### Topic extraction from CNN's article about Edward Snowden

In [7]:
result = tm.process_url('http://www.cnn.com/2013/06/10/politics/edward-snowden-profile/')
result.visualize_topic()

Success
Top 10 key words from the webpage
nsa 0.00291729
snowden 0.00291716
government 0.00291646
said 0.00291632
watched 0.00291619
video 0.0029161
replay 0.00291608
worked 0.00291602
obama 0.00291596
leak 0.00291594


PreparedData(topic_coordinates=            Freq  cluster  topics         x    y
topic                                           
4      99.839286        1       1 -0.040643  0.0
9       0.017857        1       2  0.004516 -0.0
8       0.017857        1       3  0.004519 -0.0
7       0.017857        1       4  0.004509 -0.0
6       0.017857        1       5  0.004520  0.0
5       0.017857        1       6  0.004523  0.0
3       0.017857        1       7  0.004519  0.0
2       0.017857        1       8  0.004505  0.0
1       0.017857        1       9  0.004517  0.0
0       0.017857        1      10  0.004516 -0.0, topic_info=     Category       Freq           Term      Total  loglift  logprob
term                                                                
202   Default  14.000000            nsa  14.000000  30.0000  30.0000
273   Default  13.000000        snowden  13.000000  29.0000  29.0000
253   Default   9.000000           said   9.000000  28.0000  28.0000
116   Default   7.000000     government   7.000000  27.0000  27.0000
324   Default   6.000000        watched   6.000000  26.0000  26.0000
323   Default   5.000000          watch   5.000000  25.0000  25.0000
192   Default   5.000000           must   5.000000  24.0000  24.0000
239   Default   5.000000         replay   5.000000  23.0000  23.0000
318   Default   5.000000          video   5.000000  22.0000  22.0000
120   Default   4.000000       guardian   4.000000  21.0000  21.0000
127   Default   4.000000         hawaii   4.000000  20.0000  20.0000
226   Default   4.000000        privacy   4.000000  19.0000  19.0000
336   Default   4.000000         worked   4.000000  18.0000  18.0000
161   Default   3.000000           leak   3.000000  17.0000  17.0000
329   Default   3.000000  whistleblower   3.000000  16.0000  16.0000
5     Default   3.000000            act   3.000000  15.0000  15.0000
204   Default   3.000000          obama   3.000000  14.0000  14.0000
61    Default   3.000000     contractor   3.000000  13.0000  13.0000
216   Default   3.000000        patriot   3.000000  12.0000  12.0000
135   Default   3.000000           hong   3.000000  11.0000  11.0000
227   Default   3.000000        program   3.000000  10.0000  10.0000
157   Default   3.000000           kong   3.000000   9.0000   9.0000
303   Default   3.000000           told   3.000000   8.0000   8.0000
198   Default   3.000000           news   3.000000   7.0000   7.0000
72    Default   2.000000         decide   2.000000   6.0000   6.0000
166   Default   2.000000           left   2.000000   5.0000   5.0000
289   Default   2.000000   surveillance   2.000000   4.0000   4.0000
50    Default   2.000000            cnn   2.000000   3.0000   3.0000
247   Default   2.000000          right   2.000000   2.0000   2.0000
56    Default   2.000000       computer   2.000000   1.0000   1.0000
...       ...        ...            ...        ...      ...      ...
202   Topic10   0.000291            nsa  14.206774  -2.1647  -5.8371
273   Topic10   0.000291        snowden  13.266138  -2.0962  -5.8371
116   Topic10   0.000291     government   7.622049  -1.5423  -5.8374
253   Topic10   0.000291           said   9.503413  -1.7629  -5.8374
324   Topic10   0.000291        watched   6.681391  -1.4107  -5.8375
318   Topic10   0.000291          video   5.740726  -1.2589  -5.8375
239   Topic10   0.000291         replay   5.740721  -1.2590  -5.8375
336   Topic10   0.000291         worked   4.800050  -1.0800  -5.8375
204   Topic10   0.000291          obama   3.859367  -0.8619  -5.8376
161   Topic10   0.000291           leak   3.859364  -0.8619  -5.8376
323   Topic10   0.000291          watch   5.740713  -1.2590  -5.8376
127   Topic10   0.000291         hawaii   4.800040  -1.0800  -5.8376
120   Topic10   0.000291       guardian   4.800038  -1.0801  -5.8376
135   Topic10   0.000291           hong   3.859378  -0.8619  -5.8376
289   Topic10   0.000291   surveillance   2.918699  -0.5826  -5.8376
192   Topic10   0.000291           must   5.740718  -1.25

#### Bulk topic modeling on multiple URLs with data extraction failure handling

In [8]:
url_list = [
    'https://www.nytimes.com/2018/08/18/us/politics/don-mcgahn-mueller-investigation.html?action=click&contentCollection=politics&region=rank&module=package&version=highlights&contentPlacement=1&pgtype=sectionfront',
    'https://www.quora.com/Is-it-possible-to-extract-topics-in-a-single-document',
    'http://uselsess.siteee',
    'https://www.cnet.com/reviews/google-home-review/',
    'https://www.amazon.com/gp/product/B009GQ034C?pf_rd_p=d1f45e03-8b73-4c9a-9beb-4819111bef9a&pf_rd_r=JY8Z5CD0VB6H3JT0212T'
]

In [9]:
result = tm.process_bulk_url(url_list)
result.visualize_topic()

Unable to fetch data from following URLS:
http://uselsess.siteee
https://www.amazon.com/gp/product/B009GQ034C?pf_rd_p=d1f45e03-8b73-4c9a-9beb-4819111bef9a&pf_rd_r=JY8Z5CD0VB6H3JT0212T


PreparedData(topic_coordinates=            Freq  cluster  topics         x         y
topic                                                
3      45.009866        1       1  0.233027  0.007994
0      40.475931        1       2 -0.173455  0.150053
2      14.455130        1       3 -0.107994 -0.219803
9       0.008439        1       4  0.006924  0.008820
8       0.008439        1       5  0.006911  0.008822
7       0.008439        1       6  0.006910  0.008803
6       0.008439        1       7  0.006918  0.008837
5       0.008439        1       8  0.006918  0.008832
4       0.008439        1       9  0.006914  0.008831
1       0.008439        1      10  0.006928  0.008809, topic_info=     Category       Freq           Term      Total  loglift  logprob
term                                                                
1134  Default  60.000000         google  60.000000  30.0000  30.0000
305   Default  68.000000           home  68.000000  29.0000  29.0000
399   Default  46.000000         mcgahn  46.000000  28.0000  28.0000
674   Default  38.000000          trump  38.000000  27.0000  27.0000
503   Default  33.000000      president  33.000000  26.0000  26.0000
148   Default  26.000000        counsel  26.000000  25.0000  25.0000
1330  Default  26.000000          smart  26.000000  24.0000  24.0000
748   Default  29.000000           best  29.000000  23.0000  23.0000
365   Default  22.000000         lawyer  22.000000  22.0000  22.0000
308   Default  25.000000          house  25.000000  21.0000  21.0000
707   Default  25.000000          white  25.000000  20.0000  20.0000
418   Default  19.000000        mueller  19.000000  19.0000  19.0000
336   Default  18.000000   investigator  18.000000  18.0000  18.0000
192   Default  18.000000       document  18.000000  17.0000  17.0000
335   Default  16.000000  investigation  16.000000  16.0000  16.0000
614   Default  18.000000        special  18.000000  15.0000  15.0000
794   Default  15.000000        extract  15.000000  14.0000  14.0000
1342  Default  14.000000        speaker  14.000000  13.0000  13.0000
953   Default  13.000000      assistant  13.000000  12.0000  12.0000
896   Default  14.000000          topic  14.000000  11.0000  11.0000
564   Default  13.000000           said  13.000000  10.0000  10.0000
1013  Default  11.000000           cnet  11.000000   9.0000   9.0000
720   Default  15.000000           word  15.000000   8.0000   8.0000
660   Default  13.000000           time  13.000000   7.0000   7.0000
838   Default  12.000000          model  12.000000   6.0000   6.0000
736   Default  11.000000         amazon  11.000000   5.0000   5.0000
1252  Default   9.000000          phone   9.000000   4.0000   4.0000
1107  Default   9.000000        feature   9.000000   3.0000   3.0000
993   Default   9.000000            car   9.000000   2.0000   2.0000
1398  Default   9.000000          voice   9.000000   1.0000   1.0000
...       ...        ...            ...        ...      ...      ...
674   Topic10   0.000211          trump  38.811105  -2.7422  -7.2585
399   Topic10   0.000211         mcgahn  46.153679  -2.9156  -7.2586
503   Topic10   0.000211      president  33.304170  -2.5899  -7.2592
794   Topic10   0.000211        extract  15.119925  -1.8005  -7.2595
148   Topic10   0.000211        counsel  26.879314  -2.3759  -7.2595
192   Topic10   0.000211       document  18.519543  -2.0036  -7.2597
365   Topic10   0.000211         lawyer  22.290228  -2.1889  -7.2598
707   Topic10   0.000211          white  25.944775  -2.3408  -7.2598
896   Topic10   0.000211          topic  14.465415  -1.7567  -7.2599
308   Topic10   0.000211          house  25.035290  -2.3053  -7.2600
614   Topic10   0.000211        special  18.602187  -2.0083  -7.2600
335   Topic10   0.000211  investigation  16.783249  -1.9055  -7.2601
305   Topic10   0.000211           home  68.505027  -3.3121  -7.2601
720   Topic10   0.000211           word  15.893197  -1.8511  -7.2602
838   Topic10   0.000211          model  12.119551  -1.5801  -7.26